In [1]:
from tqdm import tqdm
import re
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [3]:
# senti = pd.read_csv('../dictionary_word/dictionary_result.csv')
senti = pd.read_csv('../dictionary_word/dictionary_result.csv', encoding = 'cp949')
senti = senti.apply(lambda x : x if len(x['word']) >= 2 else np.nan, axis = 1).dropna()
senti.tail(10)

,word,polarity
27711,얼룩말,0.0
27712,천정,0.0
27713,미풍,0.0
27714,제플린,0.0
27715,풍미,2.0
27716,지퍼,-1.0
27717,수대,0.0
27719,동물원,0.0
27720,동물학의,0.0
27721,동물학,0.0


In [3]:
senti

,word,polarity
0,(-;,1.0
1,(;_;),-1.0
2,(^^),1.0
3,(^-^),1.0
4,(^^*,1.0
...,...,...
27716,지퍼,-1.0
27717,수대,0.0
27719,동물원,0.0
27720,동물학의,0.0


In [6]:
my_client = MongoClient('mongodb://localhost:27017/')
newsDB = my_client['newsDB']
nosummary = newsDB['nosummary2']
query = {
 'time': {
  '$gte': '2021.06.01'
 }
}
data = nosummary.find(query,{'_id':False, 'category':False, 'time':False, 'text_context':False, 'text_company':False, 'context_url':False})
news = pd.DataFrame(list(data))

In [7]:
news

,text_headline
0,플랫폼의 시대를 넘어서
1,ESG 누구를 위한 열풍인가
2,워싱턴포스트를 보며…
3,오해의 불씨 남긴 미디어 바우처 법안
4,사설 새 검찰총장이 ‘허수아비 노릇’ 하면 민심 역풍 맞을 것
...,...
1875579,울주군 국공립웅촌어린이집 그린리모델링공사 준공
1875580,영주시 농촌일손돕기 챌린지 성과…참여인원 163% 달성
1875581,속옷만 입고 도로 뛰어다닌다 붙잡힌 40대男 알고보니
1875582,광주 붕괴사고관련 철거업체 하드디스크 교체 등 증거인멸 적발


In [ ]:
news = news['text_headline']

In [ ]:
# news = pd.read_csv('../output/result/main.csv', encoding= 'cp949')
# news['제목'] = news['제목'].str.replace(pat=r'[A-Z]', repl=r'',regex=True)

In [ ]:
news['train'] = news.apply(lambda x: x['text_headline'] + x['text_context'], axis=1)
news

In [ ]:
news['text_headline'].iloc[29]

In [ ]:
news['train'].iloc[7]

In [ ]:
news

In [6]:
labels = []

titles = news['text_headline'].map(lambda x : re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', ' ', x))
words = senti['word'].to_numpy()
polarity = senti['polarity'].to_numpy()

print(len(words))
flag = 0
positive_flag = 0
negative_flag = 0
co = 0


for title in tqdm(titles):
    # if co == 30:
    #     break
    # co += 1
    positive_flag = 0
    nagative_flag = 0
    nomal_flag = 0
    label = 0
    for i in range(len(words)):
        position = title.rfind(words[i])
        while position != -1:
            # print(position/len(title), words[i], polarity[i] * position/len(title))
            p = position / len(title)
            if polarity[i] == 2:
                positive_flag += 2 * p
            elif polarity[i] == 1:
                positive_flag += 1 * p
            elif polarity[i] == -1:
                nagative_flag += 1 * p
            elif polarity[i] == -2:
                nagative_flag += 2 * p
            else:
                nomal_flag += 1
                
            position = title.rfind(words[i], 0 , position)


    sum_flag = positive_flag + nagative_flag
    minus_flag = positive_flag - nagative_flag
    
    if minus_flag > 0:
        if (minus_flag / sum_flag) > 0.15:
            label = 2
        else:
            label = 0
    elif minus_flag < 0:
        if (minus_flag / sum_flag) < -0.15:
            label = 1
        else:
            label = 0
    else:
        label = 0
        
    # print(title+' : '+str(label) + ' '+str(positive_flag) + ' '+str(nagative_flag))
    labels.append(label)
    
news['label'] = labels

27287


100%|████████████████████████████████████████████████████████████████████| 6091108/6091108 [13:35:18<00:00, 124.52it/s]


In [7]:
news.to_csv('./data/data3.csv', index = False)

In [ ]:
news.to_csv('./data/data2.csv', index = False)

In [8]:
js = news.to_json('./data/data3.json', orient='records',  force_ascii=False)

In [11]:
js = pd.read_json('./data/data3.json')

In [12]:
js

,text_headline,label
0,사설 北 핵·ICBM 시험 재개 시사… ‘잘못된 길’ 선택해선 안 된다,1
1,사설 수출 역성장 멈추려면 기업 활력부터 되살려내야,2
2,사설 여권 일방통행식 국회 운영 정국 정상화 가능하겠나,2
3,설왕설래 인구절벽,1
4,세계포럼 친중 정책 레드라인 넘지 말아야,2
...,...,...
6091103,고무줄 면역효과 불구 영국은 왜 아스트라제네카를 승인했나,2
6091104,박원순의 마지막 이틀 부인→고백→대응→포기 흔들렸다,2
6091105,경기도 공공배달앱 ‘배달특급’ 20여일만에 거래액 20억 돌파,2
6091106,알림뉴시스 콘텐츠 저작권 고지,0


In [13]:
my_client = MongoClient('mongodb://localhost:27017/')
newsDB = my_client['newsDB']
nosummary3 = newsDB['nosummary4']
data = nosummary3.insert_many(js)
# news = pd.DataFrame(list(data))

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().